In [4]:
from langchain import hub
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import StructuredTool


from db.db import get_session
from db.models import Grade
from dto.response.matrix_chats import MessageDict
import os
from typing import TypedDict, Annotated, Literal, List, Any, Optional

from dotenv import load_dotenv
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage, SystemMessage
from langgraph.graph.graph import CompiledGraph
from langgraph.types import interrupt, Command
from langgraph.graph import StateGraph, START, END, add_messages
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

from service.service import BaseService
from utils.common import convert_msg_dict_to_langgraph_format

load_dotenv()


LITE_LLM_API_KEY = os.getenv("OPENAI_API_KEY")
# LITE_LLM_URL = os.getenv("OPENAI_BASE_URL")
# LITE_MODEL = os.getenv("OPENAI_MODEL")

# model = ChatOpenAI(
#     model=LITE_MODEL,
#     api_key=LITE_LLM_API_KEY,
#     base_url=LITE_LLM_URL,
#     streaming=True,
#     verbose=True,
# )
model = ChatOpenAI(
    model="gpt-4o",
    api_key=LITE_LLM_API_KEY,
)


class AnswerClassification(BaseModel):
    categorization: str = Field(description="classification")
    note: str = Field(description="short additional notes from your observation")


class GuidanceState(TypedDict):
    question: str
    answer: str
    messages: Annotated[list, add_messages]
    classification: str
    irregularity_amount: Optional[int]


async def prepare_discussion(messages: list[BaseMessage]) -> str:
    discussion = ""
    print(f"Preparing discussion for {messages}")
    for msg in messages:
        if isinstance(msg, AIMessage):
            discussion += f"Question: {msg.content}\n"
        elif isinstance(msg, HumanMessage):
            discussion += f"Answer: {msg.content}\n"
    print(f"Discussion:\n{discussion}")
    return discussion


async def classify_answer(state: GuidanceState) -> GuidanceState:
    prompt_template = ChatPromptTemplate.from_messages(
        (
            "system",
            """
You are classifying discussion (question noted with "Question/Statement:" and answer noted with "Answer:" within pairs) with user to a question to one of the following categories:
direct: answer makes sense and unquestionably answers the question that was asked, without any ambiguity, confusion, evasion, make belief or contradiction
need_help: When user needs additional help with the question or is asking a question related to the question we asked
evasion: When user is evading to answer clearly a question, or when intentionally creating confusion
confusion: When user seems to be confused on the question
unknown: When other categories do not apply

Please prioritize the latest answers than the question and answers from beginning of the discussion!
Answer with the categorization without additional explanation!
Please answer directly to the user!
Original question is the most important question explaining the purpose and intent of the conversation!

Discussion:
{discussion}
        """,
        )
    )
    structured_model = model.with_structured_output(AnswerClassification)
    full_discussion = await prepare_discussion(state["messages"])
    prompt = prompt_template.format(discussion=full_discussion)

    answer = await structured_model.ainvoke(prompt)
    print("CLASSIFY ANSWER ANSWER", answer.categorization)
    msgs = [AIMessage(answer.categorization)]
    if answer.categorization == "direct":
        msgs = []
    return {
        "messages": msgs,
        "classification": answer.categorization,
        "question": None,
        "answer": answer.categorization,
        "irregularity_amount": state["irregularity_amount"],
    }


async def need_help(state: GuidanceState) -> GuidanceState:
    print("NEED HELP")
    prompt_template = ChatPromptTemplate.from_messages(
        (
            "system",
            """
            Provide additional help and explanation to the user explaining the original intent based on the entire
            Discussion.
            Discussion contains Question Answer pairs noted with question "Question/Statement:" and "Answer:"
            Give the user options and possibilities to help him get to the final answer.
            Original question is the most important question explaining the purpose and intent of the conversation!
            Please answer directly to the user!
            Here is the Discussion:
            {discussion}
            """,
        )
    )
    full_discussion = await prepare_discussion(state["messages"])
    prompt = prompt_template.format(discussion=full_discussion)

    answer = await model.ainvoke(prompt)
    print("NEED HELP ANSWER", answer.content)
    return {
        "messages": [AIMessage(answer.content)],
        "classification": state["classification"],
        "question": state["question"],
        "answer": state["answer"],
        "irregularity_amount": state["irregularity_amount"],
    }


async def evasion(state: GuidanceState) -> GuidanceState:
    print("EVASION")
    prompt_template = ChatPromptTemplate.from_messages(
        (
            "system",
            """
        The user is evading the question, providing irrelevant or unrelated answers to the questions based on the discussion.
        Please provide a user with explanation that if continued this will be escalated to managers.
        Original question is the most important question explaining the purpose and intent of the conversation!
        Please answer directly to the user!

        Here is the discussion:
        {discussion}
        """,
        )
    )
    full_discussion = await prepare_discussion(state["messages"])

    prompt = prompt_template.format(discussion=full_discussion)
    answer = await model.ainvoke(prompt)
    print("EVASION ANSWER", answer.content)
    return {
        "messages": [AIMessage(answer.content)],
        "classification": state["classification"],
        "question": state["question"],
        "answer": state["answer"],
        "irregularity_amount": state["irregularity_amount"] + 1,
    }


async def confusion(state: GuidanceState) -> GuidanceState:
    prompt_template = ChatPromptTemplate.from_messages(
        (
            "system",
            """
        The user seems to be confused on the question based on the entire discussion.
        Reiterate the question with additional details explaining better.
        Original question is the most important question explaining the purpose and intent of the conversation!
        Please answer directly to the user!
        Here is the discussion:
        {discussion}
        """,
        )
    )
    full_discussion = await prepare_discussion(state["messages"])
    prompt = prompt_template.format(discussion=full_discussion)
    answer = await model.ainvoke(prompt)
    print("CONFUSION ANSWER", answer.content)
    return {
        "messages": [AIMessage(answer.content)],
        "classification": state["classification"],
        "question": state["question"],
        "answer": state["answer"],
        "irregularity_amount": state["irregularity_amount"] + 1,
    }


async def route_to_individual_helper(
    state: GuidanceState,
) -> Literal["need_help", "evasion", "direct", "confusion"]:
    if state["classification"] == "need_help":
        return "need_help"
    elif state["classification"] == "evasion":
        return "evasion"
    elif state["classification"] == "direct":
        return "direct"
    else:
        return "confusion"


async def finish(state: GuidanceState) -> GuidanceState:
    return state


async def direct(state: GuidanceState) -> GuidanceState:
    return state


async def build_graph() -> CompiledGraph:
    classify_answers = StateGraph(GuidanceState)
    classify_answers.add_node("classify_answer", classify_answer)
    classify_answers.add_node("evasion", evasion)
    classify_answers.add_node("confusion", confusion)
    classify_answers.add_node("need_help", need_help)
    classify_answers.add_node("finish", finish)
    classify_answers.add_node("direct", direct)
    classify_answers.add_edge(START, "classify_answer")
    classify_answers.add_conditional_edges(
        "classify_answer", route_to_individual_helper
    )
    classify_answers.add_edge("need_help", "finish")
    classify_answers.add_edge("evasion", "finish")
    classify_answers.add_edge("confusion", "finish")
    classify_answers.add_edge("direct", END)
    classify_answers.add_edge("finish", END)

    return classify_answers.compile()


async def run_answer_classifier(messages: list[MessageDict]) -> dict:
    graph = await build_graph()
    messages_to_send = convert_msg_dict_to_langgraph_format(messages)
    result = await graph.ainvoke(
        {
            "messages": messages_to_send,
            "classification": "",
        }
    )
    print(result)
    return result


class GuidanceHelperStdOutput(BaseModel):
    has_user_answered: bool = Field(
        description="Whether the user has correctly answered the topic at hand"
    )
    expertise_level: str = Field(
        description="The expertise user has self evaluated himself with"
    )
    expertise_id: int = Field(description="The expertise or grade ID")
    should_admin_be_involved: bool = Field(
        description="Whether the admin should be involved if user is evading the topic or fooling around"
    )
    message: str = Field(description="Message to send to the user")


async def get_grades_or_expertise() -> List[Grade]:
    """
    Useful tool to retrieve current grades or expertise level grading system
    :return: List of json representing those grades and all their fields
    """
    async for session in get_session():
        service: BaseService[Grade, int, Any, Any] = BaseService(Grade, session)
        all_db_grades = await service.list_all()
        all_grades_json: List[str] = []
        for grade in all_db_grades:
            json_grade = grade.model_dump_json()
            all_grades_json.append(json_grade)
        return all_grades_json


async def provide_guidance(msgs: List[str]) -> GuidanceHelperStdOutput:
    tools = [
        StructuredTool.from_function(
            function=get_grades_or_expertise,
            coroutine=get_grades_or_expertise,
        )
    ]
    intermediate_steps = []

    system_msg = """
    You are helping the user to properly grade their expertise in the mentioned field.
    Everything you help him with should be done by utilizing the tools or around the topic
    of helping him populate his expertise level on the topic.
    Do not discuss anything except from the provided context.
    You are guiding the user to evaluate himself on provided topic.
    Do not discuss anything (any other topic) except from the ones provided in topic!
    Do not chat about other topics with the user, guide him how to populate his expertise with the grades provided
    Warn the user if answering with unrelated topics or evading to answer the question will be escalated by involving managers!
    Topic: {context}
    If the user is evading to answer the question and is not asking any questions related to the topic for 4 or 5 messages
    please involve admin
    When the user answers with proper categorization of skills return only that categorization!
    """
    agent = create_react_agent(
        model=model, tools=tools, response_format=GuidanceHelperStdOutput
    )
    async for chunk in agent.astream(
        {
            "messages": [SystemMessage(system_msg)] + msgs,
            "context": msgs[0],
            "intermediate_steps": intermediate_steps,
        }
    ):
        print("CHUNK GUIDANCE", chunk)
        yield chunk

In [5]:
from dto.response.grades import GradeResponseBase
from typing import List
from utils.common import convert_msg_dict_to_langgraph_format
import asyncio

grades: List[GradeResponseBase] = [
    GradeResponseBase(
        id=1,
        label="Not Informed",
        value=1
    ),
    GradeResponseBase(
        id=2,
        label="Informed Basics",
        value=2
    ),
    GradeResponseBase(
        id=3,
        label="Informed in Details",
        value=3
    ),
    GradeResponseBase(
        id=4,
        label="Practice and Lab Examples",
        value=4
    ),
    GradeResponseBase(
        id=5,
        label="Production Maintenance",
        value=5
    ),
    GradeResponseBase(
        id=6,
        label="Production from Scratch",
        value=6
    ),
    GradeResponseBase(
        id=7,
        label="Educator/Expert",
        value=7
    ),
]

msgs: List[MessageDict] = [
    MessageDict(
        msg_type="ai",
        message="""
        Expertise in Cryptography
        Welcome, Jessica! In this discussion, we will explore your expertise in Cryptography, which focuses on implementing encryption, hashing, and secure communication protocols. Understanding the appropriate expertise level is crucial for your learning and application in the field.

        We offer various expertise grades to help you identify where you stand or where you want to grow:

        Not Informed - Basic understanding of the subject.
        Informed Basics - Familiarity with fundamental concepts.
        Informed in Details - Comprehensive knowledge of the topic.
        Practice and Lab Examples - Practical experience and demonstration.
        Production Maintenance - Hands-on experience in maintaining production systems.
        Production from Scratch - Ability to build production systems from the ground up.
        Educator/Expert - Mastery of the subject, capable of teaching others.
        Select the expertise level that resonates with your current understanding or desired growth in Cryptography, and let’s enhance your skills!
        """
    ),
    MessageDict(
        msg_type="human",
        message="""
        bla bla bla
        """
    ),
    MessageDict(
        msg_type="ai",
        message="""
        It seems like the response provided doesn't address the question or contribute meaningfully to the discussion.It's important to provide relevant and thoughtful answers to ensure productive communication. If there are any concerns or confusion about the topic, please feel free to ask for clarification or more information.

If this pattern of providing unrelated answers continues, we may need to escalate the matter to managers for further assistance. Your cooperation is appreciated as we strive to make this discussion beneficial for everyone involved. Let's work together to ensure the conversation stays focused and effective.
        """
    ),
    MessageDict(
        msg_type="human",
        message="""
        Working great
        """
    ),
    MessageDict(
        msg_type="ai",
        message="""
        User,

        I noticed that the answers you've been providing are not directly addressing the questions asked and seem to be off-topic or unrelated. It's important for us to maintain clear and relevant communication to ensure that we can assist you effectively.

        If this pattern continues, we may need to escalate the issue to our managers for further review. Please let us know if there's anything we can do to help or clarify things for you.

        Thank you for your understanding and cooperation.
        """
    ),
    MessageDict(
        msg_type="human",
        message="""
        Practice and Lab Examples
        """
    ),
]

answer_classifier_response = await run_answer_classifier(msgs)
print("FULL RESPONSE", answer_classifier_response)

Preparing discussion for [AIMessage(content='\n        Expertise in Cryptography\n        Welcome, Jessica! In this discussion, we will explore your expertise in Cryptography, which focuses on implementing encryption, hashing, and secure communication protocols. Understanding the appropriate expertise level is crucial for your learning and application in the field.\n\n        We offer various expertise grades to help you identify where you stand or where you want to grow:\n\n        Not Informed - Basic understanding of the subject.\n        Informed Basics - Familiarity with fundamental concepts.\n        Informed in Details - Comprehensive knowledge of the topic.\n        Practice and Lab Examples - Practical experience and demonstration.\n        Production Maintenance - Hands-on experience in maintaining production systems.\n        Production from Scratch - Ability to build production systems from the ground up.\n        Educator/Expert - Mastery of the subject, capable of teachin

KeyError: 'irregularity_amount'

In [6]:
formatted_msgs = convert_msg_dict_to_langgraph_format(msgs)
discussion = await prepare_discussion({"messages": formatted_msgs})
print(discussion)

Preparing discussion for {'messages': [AIMessage(content='\n        Expertise in Cryptography\n        Welcome, Jessica! In this discussion, we will explore your expertise in Cryptography, which focuses on implementing encryption, hashing, and secure communication protocols. Understanding the appropriate expertise level is crucial for your learning and application in the field.\n\n        We offer various expertise grades to help you identify where you stand or where you want to grow:\n\n        Not Informed - Basic understanding of the subject.\n        Informed Basics - Familiarity with fundamental concepts.\n        Informed in Details - Comprehensive knowledge of the topic.\n        Practice and Lab Examples - Practical experience and demonstration.\n        Production Maintenance - Hands-on experience in maintaining production systems.\n        Production from Scratch - Ability to build production systems from the ground up.\n        Educator/Expert - Mastery of the subject, capab

In [7]:
class EvalLine(BaseModel):
    input: str
    output: str


with open("eval.jsonl", "r") as f:
    lines = f.readlines()
    for line in lines[0:10]:
        eval_line = EvalLine.model_validate_json(line)
        ai_msg = MessageDict(msg_type="ai", message=eval_line.input)
        human_msg = MessageDict(msg_type="human", message=eval_line.output)
        # print(eval_line)
        answer_classifier_response = await run_answer_classifier([ai_msg, human_msg])
        print("FULL RESPONSE", answer_classifier_response)

Preparing discussion for [AIMessage(content="Expertise Selection for Elasticsearch\n\nWelcome, Evelyn!\n\nToday's discussion focuses on identifying the appropriate expertise level for you in Elasticsearch, a powerful technology used widely in modern systems. The goal is to help you understand different grades of expertise, ranging from basic awareness to advanced proficiency. This understanding will guide you to select the suitable level needed to effectively manage and utilize Elasticsearch capabilities.\n\nHere are the available expertise levels:\n\nNot Informed: If you're new to Elasticsearch, start here for a foundational introduction.\nInformed Basics: Choose this if you have a basic understanding and want to learn more.\nInformed in Details: Opt for this if you're familiar with Elasticsearch and wish to delve deeper.\nPractice and Lab Examples: Good for practical learners looking to apply skills in lab settings.\nProduction Maintenance: Ideal for those maintaining existing Elasti

KeyError: 'irregularity_amount'